We have now merged the data into a full dataset and the data cleaning and transformation will start in this part
the goal here is to create various variables that can be used for our machine learning model. 

In [1]:
import pandas as pd
file_path = r"C:\Users\Kasper\OneDrive - Aarhus universitet\Dokumenter\Kandidat\2. Semester\Data Science Project\Data science project part 2\merged_dataset.csv"
# Load the dataset
Football_data = pd.read_csv(file_path)

# Optionally, verify that the data has loaded correctly by displaying the first few rows
print(Football_data.head())


    Wk  Day        Date   Time           Home   xG Score  xG.1           Away  \
0  1.0  Fri  2019-08-16  20:00  Athletic Club  0.5   1–0   0.9      Barcelona   
1  1.0  Sat  2019-08-17  16:00     Celta Vigo  1.6   1–3   2.1    Real Madrid   
2  1.0  Sat  2019-08-17  18:00       Valencia  2.3   1–1   1.8  Real Sociedad   
3  1.0  Sat  2019-08-17  19:00       Mallorca  1.6   2–1   0.7          Eibar   
4  1.0  Sat  2019-08-17  20:00        Leganés  1.4   0–1   0.2        Osasuna   

   Attendance  ... AvgC<2.5  AHCh B365CAHH  B365CAHA PCAHH  PCAHA  MaxCAHH  \
0     47693.0  ...     1.97  0.75     1.93      2.00  1.91   2.01     2.02   
1     23566.0  ...     2.63  1.00     1.82      1.97  1.85   2.07     2.00   
2     41846.0  ...     1.82 -0.75     1.94      1.99  1.92   2.00     1.96   
3     15127.0  ...     1.66  0.00     2.11      1.82  2.09   1.83     2.12   
4     10020.0  ...     1.46 -0.50     1.89      2.04  1.90   2.01     1.95   

  MaxCAHA  AvgCAHH  AvgCAHA  
0    2.03     

In [2]:
# Check the shape of the DataFrame
print("Data Shape:", Football_data.shape)


Data Shape: (1860, 115)


In [3]:
import pandas as pd

# Temporarily adjust the pandas display settings to show all columns
pd.set_option('display.max_rows', None)  # This sets the display to show all rows

# Count and print missing values in each column
missing_data_counts = Football_data.isnull().sum()
print("Missing Data Counts in Each Column:")
print(missing_data_counts)

# Reset display options back to default if desired
pd.reset_option('display.max_rows')


Missing Data Counts in Each Column:
Wk                 0
Day                0
Date               0
Time               0
Home               0
xG                 0
Score              0
xG.1               0
Away               0
Attendance       487
Venue              0
Referee            0
Match Report       0
Notes           1860
Div                0
FTHG               0
FTAG               0
FTR                0
HTHG               0
HTAG               0
HTR                0
HS                 0
AS                 0
HST                0
AST                0
HF                 0
AF                 0
HC                 0
AC                 0
HY                 0
AY                 0
HR                 0
AR                 0
B365H              0
B365D              0
B365A              0
BWH               10
BWD               10
BWA               10
IWH              150
IWD              150
IWA              150
PSH                3
PSD                3
PSA                3
WHH                

In [4]:
#We start by looking at attendance, since this has a lot of missing values due to one season being without numbers on this particular data.
#We impute the data by finding the average attendance for each team over the rest of the games and the impute the mean number

# Calculate the average attendance for each home team
average_attendance_by_home_team = Football_data.groupby('Home')['Attendance'].mean()
average_attendance_by_home_team

Home
Alavés             14665.240000
Almería            13003.055556
Athletic Club      40752.573529
Atlético Madrid    54743.898551
Barcelona          62721.231884
Betis              47637.173913
Celta Vigo         14949.942029
Cádiz              16561.800000
Eibar               6066.307692
Elche              17392.179487
Espanyol           20573.333333
Getafe             10618.507246
Girona             11988.277778
Granada            15345.244898
Huesca                      NaN
Las Palmas         25301.470588
Leganés            10231.285714
Levante            16718.125000
Mallorca           14931.927536
Osasuna            18176.782609
Rayo Vallecano     10978.963636
Real Madrid        58194.250000
Real Sociedad      30450.882353
Sevilla            33976.647059
Valencia           37192.271429
Valladolid         20498.093750
Villarreal         16026.681159
Name: Attendance, dtype: float64

In [5]:
#since huesca has only been in the best league in one season. the particular season where no attendance was counted.
#we have then found the average attendance for that season and will impute a random number that goes 20% above the average
#and 30% below the average

# Impute missing attendance values based on the home team's average attendance
Football_data['Attendance'] = Football_data.apply(
    lambda row: average_attendance_by_home_team[row['Home']] if pd.isnull(row['Attendance']) else row['Attendance'],
    axis=1
)


In [6]:
import pandas as pd
import numpy as np

# Define a function to impute attendance
def impute_huesca_attendance(row):
    if row['Home'] == 'Huesca' and pd.isnull(row['Attendance']):
        return np.random.randint(3803, 6521)
    else:
        return row['Attendance']

# Apply the function to the DataFrame
Football_data['Attendance'] = Football_data.apply(impute_huesca_attendance, axis=1)


In [7]:
# check if there is more missing values in the Attendance column
print("Missing values in Attendance after all imputations:", Football_data['Attendance'].isnull().sum())


Missing values in Attendance after all imputations: 0


In [8]:
Football_data['Date'] = pd.to_datetime(Football_data['Date'], format='%Y-%m-%d')


# Define season date ranges
seasons = {
    '19/20': ('2019-08-10', '2020-07-25'),
    '20/21': ('2020-09-05', '2021-05-30'),
    '21/22': ('2021-08-10', '2022-05-25'),
    '22/23': ('2022-08-10', '2023-06-10'),
    '23/24': ('2023-08-05', '2024-05-10')
}

# Function to determine season based on date
def determine_season(date):
    for season, (start_date, end_date) in seasons.items():
        if pd.to_datetime(start_date) <= date <= pd.to_datetime(end_date):
            return season
    return 'Unknown'  # If no season matches

# Apply the function to create a new 'Season' column
Football_data['Season'] = Football_data['Date'].apply(determine_season)

print(Football_data[['Date', 'Season']].tail())

           Date Season
1855 2024-05-05  23/24
1856 2024-05-05  23/24
1857 2024-05-05  23/24
1858 2024-05-05  23/24
1859 2024-05-05  23/24


In [9]:
# Convert Date to datetime if not already done
Football_data['Date'] = pd.to_datetime(Football_data['Date'], format='%Y-%m-%d')

# Ensure data is sorted by Date for each team to maintain match order
Football_data.sort_values(['Date'], inplace=True)

# Calculate cumulative home goals correctly for each home team grouped by seasons
Football_data['CumulativeHomeGoals'] = Football_data.groupby(['Home', 'Season'])['FTHG'].transform(lambda x: x.cumsum().shift(1).fillna(0))

# Do the same for away goals
Football_data.sort_values(['Date'], inplace=True)
Football_data['CumulativeAwayGoals'] = Football_data.groupby(['Away', 'Season'])['FTAG'].transform(lambda x: x.cumsum().shift(1).fillna(0))

# Calculate league cumulative home and away goals for reference or other calculations
Football_data['LeagueCumulativeHomeGoals'] = Football_data.groupby(['Season'])['FTHG'].cumsum().shift(1).fillna(0)
Football_data['LeagueCumulativeAwayGoals'] = Football_data.groupby(['Season'])['FTAG'].cumsum().shift(1).fillna(0)

Football_data.sort_index(inplace=True)

# You can now add attack strength calculations or other metrics as needed
# Example: Calculating attack strength based on cumulative goals
Football_data['AttackStrengthHome'] = Football_data['CumulativeHomeGoals'] / Football_data['LeagueCumulativeHomeGoals']
Football_data['AttackStrengthAway'] = Football_data['CumulativeAwayGoals'] / Football_data['LeagueCumulativeAwayGoals']

# Now you can inspect the data including the cumulative league goals
print(Football_data[['Date', 'Home', 'Away', 'FTHG', 'FTAG', 'CumulativeHomeGoals', 'LeagueCumulativeHomeGoals', 'AttackStrengthHome', 'CumulativeAwayGoals', 'LeagueCumulativeAwayGoals', 'AttackStrengthAway']].head(20))


         Date             Home             Away  FTHG  FTAG  \
0  2019-08-16    Athletic Club        Barcelona     1     0   
1  2019-08-17       Celta Vigo      Real Madrid     1     3   
2  2019-08-17         Valencia    Real Sociedad     1     1   
3  2019-08-17         Mallorca            Eibar     2     1   
4  2019-08-17          Leganés          Osasuna     0     1   
5  2019-08-17       Villarreal          Granada     4     4   
6  2019-08-18           Alavés          Levante     1     0   
7  2019-08-18         Espanyol          Sevilla     0     2   
8  2019-08-18            Betis       Valladolid     1     2   
9  2019-08-18  Atlético Madrid           Getafe     1     0   
10 2019-08-23          Granada          Sevilla     0     1   
11 2019-08-23          Levante       Villarreal     2     1   
12 2019-08-24          Osasuna            Eibar     0     0   
13 2019-08-24      Real Madrid       Valladolid     1     1   
14 2019-08-24           Getafe    Athletic Club     1  

In [10]:
# Convert Date to datetime if not already done
Football_data['Date'] = pd.to_datetime(Football_data['Date'], format='%Y-%m-%d')

# Ensure data is sorted by Date for each team to maintain match order
Football_data.sort_values(['Date'], inplace=True)

# Calculate cumulative home goals correctly for each home team grouped by seasons
Football_data['CumulativeGoalsConcededHome'] = Football_data.groupby(['Home', 'Season'])['FTAG'].transform(lambda x: x.cumsum().shift(1).fillna(0))

# Do the same for away goals
Football_data.sort_values(['Date'], inplace=True)
Football_data['CumulativeGoalsConcededAway'] = Football_data.groupby(['Away', 'Season'])['FTHG'].transform(lambda x: x.cumsum().shift(1).fillna(0))

# Calculate league cumulative home and away goals for reference or other calculations
Football_data['LeagueCumulativeGoalsConcededHome'] = Football_data.groupby(['Season'])['FTAG'].cumsum().shift(1).fillna(0)
Football_data['LeagueCumulativeGoalsConcededAway'] = Football_data.groupby(['Season'])['FTHG'].cumsum().shift(1).fillna(0)

Football_data.sort_index(inplace=True)

# You can now add attack strength calculations or other metrics as needed
# Example: Calculating attack strength based on cumulative goals
Football_data['DefenseWeaknessHome'] = Football_data['CumulativeGoalsConcededHome'] / Football_data['LeagueCumulativeGoalsConcededHome']
Football_data['DefenseWeaknessAway'] = Football_data['CumulativeGoalsConcededAway'] / Football_data['LeagueCumulativeGoalsConcededAway']

# Now you can inspect the data including the cumulative league goals
print(Football_data[['Date', 'Home', 'Away', 'FTHG', 'FTAG', 'DefenseWeaknessHome', 'CumulativeGoalsConcededHome', 'LeagueCumulativeGoalsConcededHome', 
                     'DefenseWeaknessAway', 'CumulativeGoalsConcededAway', 'LeagueCumulativeGoalsConcededAway']].head(5))

        Date           Home           Away  FTHG  FTAG  DefenseWeaknessHome  \
0 2019-08-16  Athletic Club      Barcelona     1     0                  NaN   
1 2019-08-17     Celta Vigo    Real Madrid     1     3                  NaN   
2 2019-08-17       Valencia  Real Sociedad     1     1                  0.0   
3 2019-08-17       Mallorca          Eibar     2     1                  0.0   
4 2019-08-17        Leganés        Osasuna     0     1                  0.0   

   CumulativeGoalsConcededHome  LeagueCumulativeGoalsConcededHome  \
0                          0.0                                0.0   
1                          0.0                                0.0   
2                          0.0                                3.0   
3                          0.0                                4.0   
4                          0.0                                5.0   

   DefenseWeaknessAway  CumulativeGoalsConcededAway  \
0                  NaN                          0.0   


In [11]:
# Filter the DataFrame for rows where 'Home' or 'Away' is 'Barcelona'
barcelona_games = Football_data[(Football_data['Home'] == 'Barcelona') | (Football_data['Away'] == 'Barcelona')]

# Display the first five rows of the filtered data
print(barcelona_games.head())


     Wk  Day       Date   Time           Home   xG Score  xG.1       Away  \
0   1.0  Fri 2019-08-16  20:00  Athletic Club  0.5   1–0   0.9  Barcelona   
19  2.0  Sun 2019-08-25  20:00      Barcelona  2.6   5–2   0.2      Betis   
22  3.0  Sat 2019-08-31  16:00        Osasuna  1.8   2–2   0.4  Barcelona   
34  4.0  Sat 2019-09-14  20:00      Barcelona  2.4   5–2   1.5   Valencia   
44  5.0  Sat 2019-09-21  20:00        Granada  1.9   2–0   0.5  Barcelona   

    Attendance  ... LeagueCumulativeHomeGoals LeagueCumulativeAwayGoals  \
0      47693.0  ...                       0.0                       0.0   
19     79159.0  ...                      17.0                      20.0   
22     16742.0  ...                      25.0                      23.0   
34     81617.0  ...                      44.0                      39.0   
44     18880.0  ...                      54.0                      48.0   

   AttackStrengthHome  AttackStrengthAway CumulativeGoalsConcededHome  \
0            

In [12]:

# Assign points based on match results
Football_data['HomePoints'] = Football_data['FTR'].apply(lambda x: 3 if x == 'H' else 1 if x == 'D' else 0)
Football_data['AwayPoints'] = Football_data['FTR'].apply(lambda x: 3 if x == 'A' else 1 if x == 'D' else 0)

# Ensure the DataFrame is sorted by Date for accurate cumulative calculations
Football_data.sort_values(by='Date', inplace=True)

# Calculate cumulative points for home and away teams, shifting to ensure current game is not included
Football_data['CumulativeHomePoints'] = Football_data.groupby(['Home', 'Season'])['HomePoints'].transform(lambda x: x.cumsum().shift(1).fillna(0))
Football_data['CumulativeHomeGames'] = Football_data.groupby(['Home', 'Season']).cumcount()

Football_data['CumulativeAwayPoints'] = Football_data.groupby(['Away', 'Season'])['AwayPoints'].transform(lambda x: x.cumsum().shift(1).fillna(0))
Football_data['CumulativeAwayGames'] = Football_data.groupby(['Away', 'Season']).cumcount()

# Calculate the average points for home and away teams based on cumulative data
# We replace counts of zero games with NaN to avoid division by zero, which we then fill with zero for cleanliness
Football_data['AvgHomePoints'] = Football_data['CumulativeHomePoints'] / Football_data['CumulativeHomeGames'].replace(0, np.nan)
Football_data['AvgAwayPoints'] = Football_data['CumulativeAwayPoints'] / Football_data['CumulativeAwayGames'].replace(0, np.nan)

# Fill any resulting NaNs from teams having no prior games with zero
Football_data['AvgHomePoints'] = Football_data['AvgHomePoints'].fillna(0)
Football_data['AvgAwayPoints'] = Football_data['AvgAwayPoints'].fillna(0)

Football_data.sort_index(inplace=True)

# Print the adjusted DataFrame for verification
print(Football_data[['Season', 'Date', 'Home', 'Away', 'HomePoints', 'AwayPoints', 'AvgHomePoints', 'AvgAwayPoints']].head(200))


    Season       Date           Home           Away  HomePoints  AwayPoints  \
0    19/20 2019-08-16  Athletic Club      Barcelona           3           0   
1    19/20 2019-08-17     Celta Vigo    Real Madrid           0           3   
2    19/20 2019-08-17       Valencia  Real Sociedad           1           1   
3    19/20 2019-08-17       Mallorca          Eibar           3           0   
4    19/20 2019-08-17        Leganés        Osasuna           0           3   
..     ...        ...            ...            ...         ...         ...   
195  19/20 2020-01-19       Mallorca       Valencia           3           0   
196  19/20 2020-01-19          Betis  Real Sociedad           3           0   
197  19/20 2020-01-19     Villarreal       Espanyol           0           3   
198  19/20 2020-01-19  Athletic Club     Celta Vigo           1           1   
199  19/20 2020-01-19      Barcelona        Granada           3           0   

     AvgHomePoints  AvgAwayPoints  
0         0.000

In [13]:
print(Football_data['Season'].value_counts())

Season
19/20    380
20/21    380
21/22    380
22/23    380
23/24    340
Name: count, dtype: int64


In [14]:
# Filter the DataFrame for rows where 'Home' or 'Away' is 'Barcelona'
barcelona_games = Football_data[(Football_data['Home'] == 'Barcelona') | (Football_data['Away'] == 'Barcelona')]

# Display the first five rows of the filtered data
print(barcelona_games[['Date', 'Score', 'Home', 'Away', 'DefenseWeaknessHome', 'DefenseWeaknessAway', 'FTHG', 'FTAG', 
                            'HomePoints', 'AwayPoints', 'AvgHomePoints', 'AvgAwayPoints']].head())


         Date Score           Home       Away  DefenseWeaknessHome  \
0  2019-08-16   1–0  Athletic Club  Barcelona                  NaN   
19 2019-08-25   5–2      Barcelona      Betis             0.000000   
22 2019-08-31   2–2        Osasuna  Barcelona             0.000000   
34 2019-09-14   5–2      Barcelona   Valencia             0.051282   
44 2019-09-21   2–0        Granada  Barcelona             0.020833   

    DefenseWeaknessAway  FTHG  FTAG  HomePoints  AwayPoints  AvgHomePoints  \
0                   NaN     1     0           3           0            0.0   
19             0.000000     5     2           3           0            0.0   
22             0.040000     2     2           1           1            1.0   
34             0.022727     5     2           3           0            3.0   
44             0.055556     2     0           3           0            0.0   

    AvgAwayPoints  
0             0.0  
19            0.0  
22            0.0  
34            0.0  
44        

In [15]:
# Calculate losing points for each match
Football_data['HomeLosingPoints'] = 3 - Football_data['HomePoints']
Football_data['AwayLosingPoints'] = 3 - Football_data['AwayPoints']

# Ensure the DataFrame is sorted by Date for accurate cumulative calculations
Football_data.sort_values(by='Date', inplace=True)

# Calculate cumulative losing points, excluding the current match by shifting the data
Football_data['CumulativeHomeLosingPoints'] = Football_data.groupby(['Home', 'Season'])['HomeLosingPoints'].transform(lambda x: x.cumsum().shift(1).fillna(0))
Football_data['CumulativeAwayLosingPoints'] = Football_data.groupby(['Away', 'Season'])['AwayLosingPoints'].transform(lambda x: x.cumsum().shift(1).fillna(0))

# Calculate the number of games played at home and away, excluding the current match
Football_data['CumulativeHomeGames'] = Football_data.groupby(['Home', 'Season']).cumcount().fillna(0)
Football_data['CumulativeAwayGames'] = Football_data.groupby(['Away', 'Season']).cumcount().fillna(0)

# Calculate the average losing points based on cumulative data
Football_data['AvgLosingHomePoints'] = Football_data['CumulativeHomeLosingPoints'] / Football_data['CumulativeHomeGames'].replace(0, np.nan)
Football_data['AvgLosingAwayPoints'] = Football_data['CumulativeAwayLosingPoints'] / Football_data['CumulativeAwayGames'].replace(0, np.nan)

# Fill any NaNs from division by zero (no previous games) with zero
Football_data['AvgLosingHomePoints'] = Football_data['AvgLosingHomePoints'].fillna(0)
Football_data['AvgLosingAwayPoints'] = Football_data['AvgLosingAwayPoints'].fillna(0)

Football_data.sort_index(inplace=True)

# Print the adjusted DataFrame for verification
print(Football_data[['Date', 'Home', 'Away', 'HomeLosingPoints', 'AwayLosingPoints', 'AvgLosingHomePoints', 'AvgLosingAwayPoints',
                    'CumulativeHomeLosingPoints', 'CumulativeAwayLosingPoints', 'CumulativeHomeGames', 'CumulativeAwayGames']].head(5))



        Date           Home           Away  HomeLosingPoints  \
0 2019-08-16  Athletic Club      Barcelona                 0   
1 2019-08-17     Celta Vigo    Real Madrid                 3   
2 2019-08-17       Valencia  Real Sociedad                 2   
3 2019-08-17       Mallorca          Eibar                 0   
4 2019-08-17        Leganés        Osasuna                 3   

   AwayLosingPoints  AvgLosingHomePoints  AvgLosingAwayPoints  \
0                 3                  0.0                  0.0   
1                 0                  0.0                  0.0   
2                 2                  0.0                  0.0   
3                 3                  0.0                  0.0   
4                 0                  0.0                  0.0   

   CumulativeHomeLosingPoints  CumulativeAwayLosingPoints  \
0                         0.0                         0.0   
1                         0.0                         0.0   
2                         0.0            

In [16]:
# Calculate goal difference for home and away teams in each game
Football_data['HomeGoalDiff'] = Football_data['FTHG'] - Football_data['FTAG']
Football_data['AwayGoalDiff'] = Football_data['FTAG'] - Football_data['FTHG']

# Ensure the DataFrame is sorted by Date for accurate cumulative calculations
Football_data.sort_values(by='Date', inplace=True)

# Calculate cumulative goal difference for home teams, excluding the current match by shifting the data
Football_data['CumulativeHomeGoalDiff'] = Football_data.groupby(['Home', 'Season'])['HomeGoalDiff'].transform(lambda x: x.cumsum().shift(1).fillna(0))

# Calculate cumulative away goal difference, excluding the current match by shifting the data
Football_data['CumulativeAwayGoalDiff'] = Football_data.groupby(['Away', 'Season'])['AwayGoalDiff'].transform(lambda x: x.cumsum().shift(1).fillna(0))

# Calculate the number of games played at home and away, excluding the current match
Football_data['CumulativeHomeGames'] = Football_data.groupby(['Home', 'Season']).cumcount().fillna(0)
Football_data['CumulativeAwayGames'] = Football_data.groupby(['Away', 'Season']).cumcount().fillna(0)

# Calculate the average goal difference for home and away teams based on cumulative data
Football_data['AvgGoalDiffHome'] = Football_data['CumulativeHomeGoalDiff'] / Football_data['CumulativeHomeGames'].replace(0, np.nan)  # Avoid division by zero
Football_data['AvgGoalDiffAway'] = Football_data['CumulativeAwayGoalDiff'] / Football_data['CumulativeAwayGames'].replace(0, np.nan)  # Avoid division by zero

# Fill any NaNs from division by zero (no previous games) with zero
Football_data['AvgGoalDiffHome'] = Football_data['AvgGoalDiffHome'].fillna(0)
Football_data['AvgGoalDiffAway'] = Football_data['AvgGoalDiffAway'].fillna(0)

Football_data.sort_index(inplace=True)

# Print the adjusted DataFrame for verification
print(Football_data[['Date', 'Home', 'Away', 'HomeGoalDiff', 'AwayGoalDiff', 'AvgGoalDiffHome', 'AvgGoalDiffAway',
                    'CumulativeHomeGoalDiff', 'CumulativeHomeGames', 'CumulativeAwayGoalDiff', 'CumulativeAwayGames']].head())



        Date           Home           Away  HomeGoalDiff  AwayGoalDiff  \
0 2019-08-16  Athletic Club      Barcelona             1            -1   
1 2019-08-17     Celta Vigo    Real Madrid            -2             2   
2 2019-08-17       Valencia  Real Sociedad             0             0   
3 2019-08-17       Mallorca          Eibar             1            -1   
4 2019-08-17        Leganés        Osasuna            -1             1   

   AvgGoalDiffHome  AvgGoalDiffAway  CumulativeHomeGoalDiff  \
0              0.0              0.0                     0.0   
1              0.0              0.0                     0.0   
2              0.0              0.0                     0.0   
3              0.0              0.0                     0.0   
4              0.0              0.0                     0.0   

   CumulativeHomeGames  CumulativeAwayGoalDiff  CumulativeAwayGames  
0                    0                     0.0                    0  
1                    0              

In [17]:
# Count home wins and home draws
Football_data['HomeWin'] = (Football_data['FTR'] == 'H').astype(int)
Football_data['HomeDraw'] = (Football_data['FTR'] == 'D').astype(int)

# Count away wins and away draws
Football_data['AwayWin'] = (Football_data['FTR'] == 'A').astype(int)
Football_data['AwayDraw'] = (Football_data['FTR'] == 'D').astype(int)


In [18]:
# Calculate cumulative home wins, draws, and games, then shift to exclude the current match
Football_data['CumulativeHomeWins'] = Football_data.groupby(['Home', 'Season'])['HomeWin'].transform(lambda x: x.cumsum().shift(1).fillna(0))
Football_data['CumulativeHomeDraws'] = Football_data.groupby(['Home', 'Season'])['HomeDraw'].transform(lambda x: x.cumsum().shift(1).fillna(0))
Football_data['CumulativeHomeGames'] = Football_data.groupby(['Home', 'Season']).cumcount()

# Calculate the ratios using the cumulative counts
Football_data['HomeWinsRatio'] = Football_data['CumulativeHomeWins'] / Football_data['CumulativeHomeGames'].replace(0, np.nan)
Football_data['HomeDrawsRatio'] = Football_data['CumulativeHomeDraws'] / Football_data['CumulativeHomeGames'].replace(0, np.nan) 

# Calculate cumulative away wins, draws, and games, then shift to exclude the current match
Football_data['CumulativeAwayWins'] = Football_data.groupby(['Away', 'Season'])['AwayWin'].transform(lambda x: x.cumsum().shift(1).fillna(0))
Football_data['CumulativeAwayDraws'] = Football_data.groupby(['Away', 'Season'])['AwayDraw'].transform(lambda x: x.cumsum().shift(1).fillna(0))
Football_data['CumulativeAwayGames'] = Football_data.groupby(['Away', 'Season']).cumcount()

# Calculate the ratios using the cumulative counts
Football_data['AwayWinsRatio'] = Football_data['CumulativeAwayWins'] / Football_data['CumulativeAwayGames'].replace(0, np.nan)  
Football_data['AwayDrawsRatio'] = Football_data['CumulativeAwayDraws'] / Football_data['CumulativeAwayGames'].replace(0, np.nan)  

# Optionally print some data to verify
print(Football_data[['Date', 'Home', 'Away', 'HomeWinsRatio', 'HomeDrawsRatio', 'AwayWinsRatio', 'AwayDrawsRatio',
                    'CumulativeHomeWins', 'CumulativeHomeDraws', 'CumulativeHomeGames', 'CumulativeAwayWins', 'CumulativeAwayDraws', 'CumulativeAwayGames']].head())


        Date           Home           Away  HomeWinsRatio  HomeDrawsRatio  \
0 2019-08-16  Athletic Club      Barcelona            NaN             NaN   
1 2019-08-17     Celta Vigo    Real Madrid            NaN             NaN   
2 2019-08-17       Valencia  Real Sociedad            NaN             NaN   
3 2019-08-17       Mallorca          Eibar            NaN             NaN   
4 2019-08-17        Leganés        Osasuna            NaN             NaN   

   AwayWinsRatio  AwayDrawsRatio  CumulativeHomeWins  CumulativeHomeDraws  \
0            NaN             NaN                 0.0                  0.0   
1            NaN             NaN                 0.0                  0.0   
2            NaN             NaN                 0.0                  0.0   
3            NaN             NaN                 0.0                  0.0   
4            NaN             NaN                 0.0                  0.0   

   CumulativeHomeGames  CumulativeAwayWins  CumulativeAwayDraws  \
0      

In [19]:
# Filter the DataFrame for rows where 'Home' or 'Away' is 'Barcelona'
barcelona_games = Football_data[(Football_data['Home'] == 'Barcelona') | (Football_data['Away'] == 'Barcelona')]

# Display the first five rows of the filtered data
print(barcelona_games[['Date', 'Home', 'Away', 'FTR', 'HomeWinsRatio', 'HomeDrawsRatio', 'AwayWinsRatio', 'AwayDrawsRatio',
                    'CumulativeHomeWins', 'CumulativeHomeDraws', 'CumulativeHomeGames', 'CumulativeAwayWins', 'CumulativeAwayDraws', 'CumulativeAwayGames']].head(10))


          Date           Home        Away FTR  HomeWinsRatio  HomeDrawsRatio  \
0   2019-08-16  Athletic Club   Barcelona   H            NaN             NaN   
19  2019-08-25      Barcelona       Betis   H            NaN             NaN   
22  2019-08-31        Osasuna   Barcelona   D       0.000000        1.000000   
34  2019-09-14      Barcelona    Valencia   H       1.000000        0.000000   
44  2019-09-21        Granada   Barcelona   H       0.000000        0.000000   
52  2019-09-24      Barcelona  Villarreal   H       1.000000        0.000000   
62  2019-09-28         Getafe   Barcelona   A       0.333333        0.666667   
79  2019-10-06      Barcelona     Sevilla   H       1.000000        0.000000   
81  2019-10-19          Eibar   Barcelona   A       0.666667        0.000000   
100 2019-10-29      Barcelona  Valladolid   H       1.000000        0.000000   

     AwayWinsRatio  AwayDrawsRatio  CumulativeHomeWins  CumulativeHomeDraws  \
0              NaN             NaN      

In [20]:
# Sort the DataFrame by home team and date for home calculations
Football_data.sort_values(by=['Home', 'Date'], inplace=True)

# Home team statistics
Football_data['AvgHomeShotsLast5'] = Football_data.groupby('Home')['HS'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())
Football_data['AvgHomeShotsOnTargetLast5'] = Football_data.groupby('Home')['HST'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())

# Now sort by away team and date for away calculations
Football_data.sort_values(by=['Away', 'Date'], inplace=True)

# Away team statistics
Football_data['AvgAwayShotsLast5'] = Football_data.groupby('Away')['AS'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())
Football_data['AvgAwayShotsOnTargetLast5'] = Football_data.groupby('Away')['AST'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())

# Sort the DataFrame by index to restore original order
Football_data.sort_index(inplace=True)

# Verify the first few rows to ensure the new columns are correctly calculated
print(Football_data[['Date', 'Home', 'Away', 'HS', 'AS', 'HST', 'AST', 'AvgHomeShotsLast5', 'AvgHomeShotsOnTargetLast5', 'AvgAwayShotsLast5', 'AvgAwayShotsOnTargetLast5']].tail(20))


           Date             Home             Away  HS  AS  HST  AST  \
1840 2024-04-26    Real Sociedad      Real Madrid   9   5    3    3   
1841 2024-04-27       Las Palmas           Girona  10   6    5    3   
1842 2024-04-27          Almería           Getafe  21   8   11    5   
1843 2024-04-27           Alavés       Celta Vigo  16   4    8    0   
1844 2024-04-27  Atlético Madrid    Athletic Club   8   7    3    1   
1845 2024-04-28            Cádiz         Mallorca  10   9    2    2   
1846 2024-04-28          Granada          Osasuna  12  11    4    2   
1847 2024-04-28       Villarreal   Rayo Vallecano  15  20    7    4   
1848 2024-04-28            Betis          Sevilla  15  10    5    1   
1849 2024-04-29        Barcelona         Valencia  27   5   13    3   
1850 2024-05-03           Getafe    Athletic Club  34   7    9    2   
1851 2024-05-04    Real Sociedad       Las Palmas   9   7    4    1   
1852 2024-05-04      Real Madrid            Cádiz  19   7    9    2   
1853 2

In [21]:
# Filter the DataFrame for rows where 'Home' or 'Away' is 'Barcelona'
barcelona_games = Football_data[(Football_data['Home'] == 'Barcelona') | (Football_data['Away'] == 'Barcelona')]

# Display the first five rows of the filtered data
print(barcelona_games[['Date', 'Home', 'Away', 'HS', 'AS', 'HST', 'AST', 'AvgHomeShotsLast5', 
                       'AvgHomeShotsOnTargetLast5', 'AvgAwayShotsLast5', 'AvgAwayShotsOnTargetLast5']].head(6))

         Date           Home        Away  HS  AS  HST  AST  AvgHomeShotsLast5  \
0  2019-08-16  Athletic Club   Barcelona  11  11    5    2                NaN   
19 2019-08-25      Barcelona       Betis  20   4    7    3                NaN   
22 2019-08-31        Osasuna   Barcelona  15   8    5    4                6.0   
34 2019-09-14      Barcelona    Valencia  17   9   10    3               20.0   
44 2019-09-21        Granada   Barcelona   9   8    4    1               12.0   
52 2019-09-24      Barcelona  Villarreal   8  11    4    3               18.5   

    AvgHomeShotsOnTargetLast5  AvgAwayShotsLast5  AvgAwayShotsOnTargetLast5  
0                         NaN                NaN                        NaN  
19                        NaN                NaN                        NaN  
22                        2.0               11.0                        2.0  
34                        7.0                9.0                        1.0  
44                        0.0             

In [22]:
# Sort the DataFrame by team and date to ensure correct rolling calculations
Football_data.sort_values(by=['Home', 'Date', 'Time'], inplace=True)

# Calculate rolling average of corners for home teams over the last 5 home games
Football_data['AvgHomeCornersLast5'] = Football_data.groupby('Home')['HC'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())

# Now sort by away team and date for away calculations
Football_data.sort_values(by=['Away', 'Date', 'Time'], inplace=True)

# Calculate rolling average of corners for away teams over the last 5 away games
Football_data['AvgAwayCornersLast5'] = Football_data.groupby('Away')['AC'].transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())

# Sort the DataFrame by index
Football_data.sort_index(inplace=True)

# Verify the rolling averages by displaying relevant columns
print(Football_data[['Date', 'Home', 'Away', 'HC', 'AC', 'AvgHomeCornersLast5', 'AvgAwayCornersLast5']].tail())

           Date            Home        Away  HC  AC  AvgHomeCornersLast5  \
1855 2024-05-05         Osasuna       Betis   8   5                  5.6   
1856 2024-05-05      Celta Vigo  Villarreal   2   1                  5.0   
1857 2024-05-05        Valencia      Alavés   5   7                  4.8   
1858 2024-05-05  Rayo Vallecano     Almería   9   2                  4.0   
1859 2024-05-05         Sevilla     Granada   5   4                  2.6   

      AvgAwayCornersLast5  
1855                  5.4  
1856                  3.6  
1857                  5.0  
1858                  2.8  
1859                  2.8  


We now have the general dataframe, we need to incorporate some elo ratings for the team 
so that we also rely on data that generally determines the form of the team

In [23]:
elo_data = pd.read_csv(r"C:\Users\Kasper\OneDrive - Aarhus universitet\Dokumenter\Kandidat\2. Semester\Data Science Project\Data science project part 2\team_elo.csv")

In [24]:
# Display the first few rows of the Elo data to understand its structure
print(elo_data.head())

# Check the unique team names in both DataFrames to ensure they match
print(elo_data['team'].unique())
print(Football_data['Home'].unique())


   Unnamed: 0  rank         team country  level          elo          to
0           0  11.0  Real Madrid     ESP      1  1840.553467  2019-08-01
1           1  11.0  Real Madrid     ESP      1  1840.568726  2019-08-08
2           2  11.0  Real Madrid     ESP      1  1841.599976  2019-08-15
3           3  11.0  Real Madrid     ESP      1  1841.848999  2019-08-17
4           4  11.0  Real Madrid     ESP      1  1850.097168  2019-08-22
['Real Madrid' 'Barcelona' 'Atletico' 'Girona' 'Sociedad' 'Bilbao'
 'Villarreal' 'Betis' 'Sevilla' 'Valencia' 'Getafe' 'Celta' 'Alaves'
 'Mallorca' 'Osasuna' 'Rayo Vallecano' 'Valladolid' 'Cadiz' 'Las Palmas'
 'Espanyol' 'Elche' 'Granada' 'Levante' 'Huesca' 'Eibar' 'Leganes'
 'Almeria']
['Athletic Club' 'Celta Vigo' 'Valencia' 'Mallorca' 'Leganés' 'Villarreal'
 'Alavés' 'Espanyol' 'Betis' 'Atlético Madrid' 'Granada' 'Levante'
 'Osasuna' 'Real Madrid' 'Getafe' 'Barcelona' 'Sevilla' 'Real Sociedad'
 'Eibar' 'Valladolid' 'Cádiz' 'Huesca' 'Elche' 'Rayo Valleca

In [25]:
elo_teams = {'Real Madrid', 'Barcelona', 'Atletico', 'Girona', 'Sociedad', 'Bilbao',
             'Villarreal', 'Betis', 'Sevilla', 'Valencia', 'Getafe', 'Celta', 'Alaves', 
             'Mallorca', 'Osasuna', 'Rayo Vallecano', 'Valladolid', 'Cadiz', 'Las Palmas',
             'Espanyol', 'Elche', 'Granada', 'Levante', 'Huesca', 'Eibar', 'Leganes', 'Almeria'}

football_data_teams = {'Athletic Club', 'Celta Vigo', 'Valencia', 'Mallorca', 'Leganés', 
                       'Villarreal', 'Alavés', 'Espanyol', 'Betis', 'Atlético Madrid', 
                       'Granada', 'Levante', 'Osasuna', 'Real Madrid', 'Getafe', 'Barcelona', 
                       'Sevilla', 'Real Sociedad', 'Eibar', 'Valladolid', 'Cádiz', 'Huesca', 
                       'Elche', 'Rayo Vallecano', 'Almería', 'Girona', 'Las Palmas'}

# Find differences between the two sets
diff1 = elo_teams.difference(football_data_teams)
diff2 = football_data_teams.difference(elo_teams)

# Print differences and unique team counts
print("Teams in elo_teams not in football_data_teams:", diff1)
print("Teams in football_data_teams not in elo_teams:", diff2)


Teams in elo_teams not in football_data_teams: {'Alaves', 'Almeria', 'Leganes', 'Sociedad', 'Cadiz', 'Bilbao', 'Atletico', 'Celta'}
Teams in football_data_teams not in elo_teams: {'Cádiz', 'Atlético Madrid', 'Almería', 'Alavés', 'Athletic Club', 'Real Sociedad', 'Celta Vigo', 'Leganés'}


In [26]:
# Check the count of unique team names in each set
print("Unique names in elo_teams:", len(elo_teams))
print("Unique names in football_data_teams:", len(football_data_teams))


Unique names in elo_teams: 27
Unique names in football_data_teams: 27


In [27]:
elo_data['to'] = pd.to_datetime(elo_data['to'], format='%Y-%m-%d')

In [28]:
name_mapping = {
    'Bilbao': 'Athletic Club',
    'Celta': 'Celta Vigo',
    'Valencia': 'Valencia',
    'Mallorca': 'Mallorca',
    'Leganes': 'Leganés',
    'Villarreal': 'Villarreal',
    'Alaves': 'Alavés',
    'Espanyol': 'Espanyol',
    'Betis': 'Betis',
    'Atletico': 'Atlético Madrid',
    'Granada': 'Granada',
    'Levante': 'Levante',
    'Osasuna': 'Osasuna',
    'Real Madrid': 'Real Madrid',
    'Getafe': 'Getafe',
    'Barcelona': 'Barcelona',
    'Sevilla': 'Sevilla',
    'Sociedad': 'Real Sociedad',
    'Eibar': 'Eibar',
    'Valladolid': 'Valladolid',
    'Cadiz': 'Cádiz',
    'Huesca': 'Huesca',
    'Elche': 'Elche',
    'Rayo Vallecano': 'Rayo Vallecano',
    'Almeria': 'Almería',
    'Girona': 'Girona',
    'Las Palmas': 'Las Palmas'
}

# Apply the mapping to standardize team names in elo_data
elo_data['team'] = elo_data['team'].map(name_mapping)

In [29]:
# Merge Elo ratings for home teams
Football_data = Football_data.merge(
    elo_data, 
    how='left', 
    left_on=['Home', 'Date'], 
    right_on=['team', 'to'], 
    suffixes=('', '_home')
)

# Merge Elo ratings for away teams
Football_data = Football_data.merge(
    elo_data, 
    how='left', 
    left_on=['Away', 'Date'], 
    right_on=['team', 'to'], 
    suffixes=('', '_away')
)

In [30]:
# Drop redundant columns from the merge
Football_data.drop(columns=['team', 'to', 'team_away', 'to_away'], errors='ignore', inplace=True)

# Verify the merge and the structure of the DataFrame
print(Football_data.head())


    Wk  Day       Date   Time           Home   xG Score  xG.1           Away  \
0  1.0  Fri 2019-08-16  20:00  Athletic Club  0.5   1–0   0.9      Barcelona   
1  1.0  Sat 2019-08-17  16:00     Celta Vigo  1.6   1–3   2.1    Real Madrid   
2  1.0  Sat 2019-08-17  18:00       Valencia  2.3   1–1   1.8  Real Sociedad   
3  1.0  Sat 2019-08-17  19:00       Mallorca  1.6   2–1   0.7          Eibar   
4  1.0  Sat 2019-08-17  20:00        Leganés  1.4   0–1   0.2        Osasuna   

   Attendance  ... Unnamed: 0  rank country  level          elo  \
0     47693.0  ...       2348  38.0     ESP      1  1722.019409   
1     23566.0  ...       5726  60.0     ESP      1  1681.163452   
2     41846.0  ...       4547  15.0     ESP      1  1813.655029   
3     15127.0  ...       6895   NaN     ESP      1  1566.961060   
4     10020.0  ...      13512  62.0     ESP      1  1670.794800   

   Unnamed: 0_away  rank_away country_away  level_away     elo_away  
0              422        3.0          ESP    

In [31]:
# Print all column names as a list
print(list(Football_data.columns))

['Wk', 'Day', 'Date', 'Time', 'Home', 'xG', 'Score', 'xG.1', 'Away', 'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes', 'Div', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'S

In [32]:
# Sort data for home
Football_data.sort_values(by=['Home', 'Date'], inplace=True)

# Calculate rolling mean for home team points and opponent Elo ratings excluding the current game
Football_data['RollingPointsHome'] = Football_data.groupby('Home')['HomePoints'].transform(
    lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())
Football_data['RollingOppEloHome'] = Football_data.groupby('Home')['elo_away'].transform(
    lambda x: x.rolling(window=5, min_periods=1).mean())

# Replace NaN values with 0 directly in the assignment
Football_data['RollingPointsHome'] = Football_data['RollingPointsHome'].fillna(0)
Football_data['RollingOppEloHome'] = Football_data['RollingOppEloHome'].fillna(0)

# Sort data for Away
Football_data.sort_values(by=['Away', 'Date'], inplace=True)

# Calculate rolling mean for away team points and home Elo ratings excluding the current game
Football_data['RollingPointsAway'] = Football_data.groupby('Away')['AwayPoints'].transform(
    lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())
Football_data['RollingOppEloAway'] = Football_data.groupby('Away')['elo'].transform(
    lambda x: x.rolling(window=5, min_periods=1).mean())

# Replace NaN values with 0 directly in the assignment
Football_data['RollingPointsAway'] = Football_data['RollingPointsAway'].fillna(0)
Football_data['RollingOppEloAway'] = Football_data['RollingOppEloAway'].fillna(0)

# Calculate form scores for home and away teams
Football_data['FormHomeTeam'] = Football_data['RollingPointsHome'] * Football_data['RollingOppEloHome']
Football_data['FormAwayTeam'] = Football_data['RollingPointsAway'] * Football_data['RollingOppEloAway']


In [33]:
# Calculate form scores for home and away teams
Football_data['FormHomeTeam'] = Football_data['RollingPointsHome'] * Football_data['RollingOppEloHome']
Football_data['FormAwayTeam'] = Football_data['RollingPointsAway'] * Football_data['RollingOppEloAway']


In [34]:
Football_data.sort_index(inplace=True)

In [35]:
# Print the specified columns to check their values
print(Football_data[['Date', 'Home', 'Away', 
                     'HomePoints', 'AwayPoints', 
                     'RollingPointsHome', 'RollingPointsAway', 
                     'RollingOppEloHome', 'RollingOppEloAway', 
                     'FormHomeTeam', 'FormAwayTeam']].head(30))


         Date             Home             Away  HomePoints  AwayPoints  \
0  2019-08-16    Athletic Club        Barcelona           3           0   
1  2019-08-17       Celta Vigo      Real Madrid           0           3   
2  2019-08-17         Valencia    Real Sociedad           1           1   
3  2019-08-17         Mallorca            Eibar           3           0   
4  2019-08-17          Leganés          Osasuna           0           3   
5  2019-08-17       Villarreal          Granada           1           1   
6  2019-08-18           Alavés          Levante           3           0   
7  2019-08-18         Espanyol          Sevilla           0           3   
8  2019-08-18            Betis       Valladolid           0           3   
9  2019-08-18  Atlético Madrid           Getafe           3           0   
10 2019-08-23          Granada          Sevilla           0           3   
11 2019-08-23          Levante       Villarreal           3           0   
12 2019-08-24          Os

We have now scraped and calculated most of the variables we need. lastly we will calculate the propabilities of
both draw, win and lose for the current game. 

In [36]:
# Constants
HomeFieldAdvantage = 80
eulers_number = np.e  

def calculate_draw_probability(HomeElo, AwayElo, HomeFieldAdvantage=80):
    eulers_number = np.e
    dr = (HomeElo + HomeFieldAdvantage - AwayElo)
    normalized_difference = dr / 200
    squared_difference = normalized_difference ** 2
    variance = 2 * (eulers_number ** 2) 
    exponent = -(squared_difference) / variance
    probability_draw = (1 / (np.sqrt(2 * np.pi * eulers_number))) * np.exp(exponent)
    return probability_draw

# Example Usage
HomeElo = 0
AwayElo = 0
print("Probability of draw:", calculate_draw_probability(HomeElo, AwayElo))




Probability of draw: 0.239365081267582


In [37]:
# Apply the function to each match in your DataFrame
Football_data['ProbabilityDraw'] = Football_data.apply(
    lambda x: calculate_draw_probability(x['elo'], x['elo_away']), axis=1
)

# Check the first few probabilities calculated
print(Football_data[['Date', 'Home', 'Away', 'elo', 'elo_away', 'ProbabilityDraw']].head())

        Date           Home           Away          elo     elo_away  \
0 2019-08-16  Athletic Club      Barcelona  1722.019409  2001.228638   
1 2019-08-17     Celta Vigo    Real Madrid  1681.163452  1841.848999   
2 2019-08-17       Valencia  Real Sociedad  1813.655029  1719.431152   
3 2019-08-17       Mallorca          Eibar  1566.961060  1712.778198   
4 2019-08-17        Leganés        Osasuna  1670.794800  1641.649048   

   ProbabilityDraw  
0         0.226260  
1         0.239320  
2         0.229859  
3         0.240204  
4         0.237143  


In [38]:
def calculate_win_probability(HomeElo, AwayElo, HomeFieldAdvantage, prob_draw):
    exponent = (-(HomeElo + HomeFieldAdvantage - AwayElo) / 400)
    win_probability_raw = 1 / (1 + 10 ** exponent)
    win_probability_adjusted = win_probability_raw - 0.5 * prob_draw
    return win_probability_adjusted

# Example Elo ratings and previously calculated probability of a draw
HomeElo = 1722.019409
AwayElo = 2001.228638
prob_draw = 0.22625962936401717

# Calculate win probability
win_probability = calculate_win_probability(HomeElo, AwayElo, HomeFieldAdvantage, prob_draw)
print("Probability of Home Win:", win_probability)
print(HomeFieldAdvantage)

Probability of Home Win: 0.12795513236041742
80


In [39]:
# Apply the function to each match in your DataFrame
Football_data['ProbabilityHomeWin'] = Football_data.apply(
    lambda x: calculate_win_probability(x['elo'], x['elo_away'], HomeFieldAdvantage, x['ProbabilityDraw']), axis=1
)

# Check the first few probabilities calculated
print(Football_data[['Date', 'Home', 'Away', 'elo', 'elo_away', 'ProbabilityDraw', 'ProbabilityHomeWin']].head(20))

         Date             Home             Away          elo     elo_away  \
0  2019-08-16    Athletic Club        Barcelona  1722.019409  2001.228638   
1  2019-08-17       Celta Vigo      Real Madrid  1681.163452  1841.848999   
2  2019-08-17         Valencia    Real Sociedad  1813.655029  1719.431152   
3  2019-08-17         Mallorca            Eibar  1566.961060  1712.778198   
4  2019-08-17          Leganés          Osasuna  1670.794800  1641.649048   
5  2019-08-17       Villarreal          Granada  1732.859985  1599.757568   
6  2019-08-18           Alavés          Levante  1683.407104  1685.884766   
7  2019-08-18         Espanyol          Sevilla  1743.833374  1748.584717   
8  2019-08-18            Betis       Valladolid  1686.496582  1617.353394   
9  2019-08-18  Atlético Madrid           Getafe  1878.425903  1761.223267   
10 2019-08-23          Granada          Sevilla  1606.124878  1763.424927   
11 2019-08-23          Levante       Villarreal  1680.713745  1728.171631   

In [40]:
def calculate_lose_probability(HomeElo, AwayElo, HomeFieldAdvantage, prob_draw):
    exponent = ((HomeElo + HomeFieldAdvantage - AwayElo) / 400)
    win_probability_raw = 1 / (1 + 10 ** exponent)
    win_probability_adjusted = win_probability_raw - 0.5 * prob_draw
    return win_probability_adjusted

# Example Elo ratings and previously calculated probability of a draw
HomeElo = 0
AwayElo = 0
prob_draw = 0.239365081267582

# Calculate win probability
lose_probability = calculate_lose_probability(HomeElo, AwayElo, HomeFieldAdvantage, prob_draw)
print("Probability of Home Lose:", lose_probability)

Probability of Home Lose: 0.2671806392130659


In [41]:
# Apply the function to each match in your DataFrame
Football_data['ProbabilityAwayWin'] = Football_data.apply(
    lambda x: calculate_lose_probability(x['elo'], x['elo_away'], HomeFieldAdvantage, x['ProbabilityDraw']), axis=1
)

# Check the first few probabilities calculated
print(Football_data[['Date', 'Home', 'Away', 'elo', 'elo_away', 'ProbabilityDraw', 'ProbabilityHomeWin', 'ProbabilityAwayWin',
                    'FormHomeTeam', 'FormAwayTeam', 
                     'RollingPointsHome', 'RollingPointsAway', 
                     'RollingOppEloHome', 'RollingOppEloAway',]].tail(20))

           Date             Home             Away          elo     elo_away  \
1840 2024-04-26    Real Sociedad      Real Madrid  1764.274780  1978.852417   
1841 2024-04-27       Las Palmas           Girona  1597.429810  1767.412720   
1842 2024-04-27          Almería           Getafe  1536.376221  1657.991333   
1843 2024-04-27           Alavés       Celta Vigo  1609.966187  1639.291870   
1844 2024-04-27  Atlético Madrid    Athletic Club  1812.163574  1756.184692   
1845 2024-04-28            Cádiz         Mallorca  1588.536621  1628.670898   
1846 2024-04-28          Granada          Osasuna  1547.670532  1643.860962   
1847 2024-04-28       Villarreal   Rayo Vallecano  1717.258911  1627.501099   
1848 2024-04-28            Betis          Sevilla  1698.196777  1693.090576   
1849 2024-04-29        Barcelona         Valencia  1868.039429  1690.780273   
1850 2024-05-03           Getafe    Athletic Club  1668.619385  1750.207031   
1851 2024-05-04    Real Sociedad       Las Palmas  1

In [42]:
# Given Elo ratings
HomeElo = 1722.019409
AwayElo = 2001.228638
HomeFieldAdvantage = 80

# Calculate probabilities using the defined functions
prob_draw = calculate_draw_probability(HomeElo, AwayElo, HomeFieldAdvantage)
prob_win = calculate_win_probability(HomeElo, AwayElo, HomeFieldAdvantage, prob_draw)
prob_lose = calculate_lose_probability(HomeElo, AwayElo, HomeFieldAdvantage, prob_draw)

prob_draw, prob_win, prob_lose


(0.22625962936401717, 0.12795513236041742, 0.6457852382755656)

In [43]:
# Print all column names as a list
print(list(Football_data.columns))

['Wk', 'Day', 'Date', 'Time', 'Home', 'xG', 'Score', 'xG.1', 'Away', 'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes', 'Div', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'S

In [44]:
columns_to_keep = [
    'Wk', 'Day', 'Date', 'Time', 'Home', 'Away', 'FTR', 'Attendance', 
    'B365H', 'B365D', 'B365A', 'AttackStrengthHome', 'AttackStrengthAway', 
    'DefenseWeaknessHome', 'DefenseWeaknessAway', 'Season', 'AvgHomePoints', 'AvgAwayPoints', 
    'AvgLosingHomePoints', 'AvgLosingAwayPoints', 'AvgGoalDiffHome', 
    'AvgGoalDiffAway', 'HomeWinsRatio', 'HomeDrawsRatio', 'AwayWinsRatio', 'AwayDrawsRatio', 
    'AvgHomeCornersLast5', 'AvgAwayCornersLast5', 'AvgHomeShotsLast5', 'AvgHomeShotsOnTargetLast5', 
    'AvgAwayShotsLast5', 'AvgAwayShotsOnTargetLast5', 'elo', 'elo_away','FormHomeTeam', 
    'FormAwayTeam', 'ProbabilityDraw', 'ProbabilityHomeWin', 'ProbabilityAwayWin'
]

# Subset the DataFrame 
Football_data = Football_data[columns_to_keep]

Football_data = Football_data.fillna(0)

print(Football_data.head(25))

     Wk  Day       Date   Time             Home             Away FTR  \
0   1.0  Fri 2019-08-16  20:00    Athletic Club        Barcelona   H   
1   1.0  Sat 2019-08-17  16:00       Celta Vigo      Real Madrid   A   
2   1.0  Sat 2019-08-17  18:00         Valencia    Real Sociedad   D   
3   1.0  Sat 2019-08-17  19:00         Mallorca            Eibar   H   
4   1.0  Sat 2019-08-17  20:00          Leganés          Osasuna   A   
5   1.0  Sat 2019-08-17  20:00       Villarreal          Granada   D   
6   1.0  Sun 2019-08-18  16:00           Alavés          Levante   H   
7   1.0  Sun 2019-08-18  18:00         Espanyol          Sevilla   A   
8   1.0  Sun 2019-08-18  20:00            Betis       Valladolid   A   
9   1.0  Sun 2019-08-18  21:00  Atlético Madrid           Getafe   H   
10  2.0  Fri 2019-08-23  19:00          Granada          Sevilla   A   
11  2.0  Fri 2019-08-23  21:00          Levante       Villarreal   H   
12  2.0  Sat 2019-08-24  16:00          Osasuna            Eibar

In [45]:
Football_data.shape

(1860, 39)

In [46]:
# Save the DataFrame to CSV
Football_data.to_csv(r"C:\Users\Kasper\OneDrive - Aarhus universitet\Dokumenter\Kandidat\2. Semester\Data Science Project\Data science project part 2\Football_data.csv", index=False)



   ---------------------------------------- 0.0/29.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/29.4 MB 393.8 kB/s eta 0:01:15
   ---------------------------------------- 0.1/29.4 MB 804.6 kB/s eta 0:00:37
    --------------------------------------- 0.4/29.4 MB 1.9 MB/s eta 0:00:16
    --------------------------------------- 0.5/29.4 MB 2.2 MB/s eta 0:00:14
   - -------------------------------------- 0.9/29.4 MB 3.5 MB/s eta 0:00:09
   - -------------------------------------- 1.2/29.4 MB 3.6 MB/s eta 0:00:08
   - -------------------------------------- 1.4/29.4 MB 3.7 MB/s eta 0:00:08
   -- ------------------------------------- 1.8/29.4 MB 4.2 MB/s eta 0:00:07
   -- ------------------------------------- 2.0/29.4 MB 4.3 MB/s eta 0:00:07
   -- ------------------------------------- 2.2/29.4 MB 4.2 MB/s eta 0:00:07
   --- --------